In [2]:
from libs.alibaba.alibaba import Alibaba

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select

from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import traceback
import types

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [4]:
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
alibaba_eyelash = Alibaba(lid, lpwd, browser=browser)
alibaba_eyelash.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [8]:
products = alibaba_eyelash.get_posted_product_info(99)

primary 打开 产品管理 页面 ... ...
primary 切换 至 显示全部产品 ... ...
primary 查找 全部产品列表 第 1 页
primary 完成 查找！
primary 查找 全部产品列表 第 2 页
primary 完成 查找！
primary 查找 全部产品列表 第 3 页
primary 完成 查找！
primary 查找 全部产品列表 第 4 页
primary 完成 查找！
primary 查找 全部产品列表 第 5 页
primary 完成 查找！
primary 查找 全部产品列表 第 6 页
primary 完成 查找！
primary 查找 全部产品列表 第 7 页
primary 完成 查找！
primary 查找 全部产品列表 第 8 页
primary 完成 查找！
primary 查找 全部产品列表 第 9 页
primary 完成 查找！
primary 查找 全部产品列表 第 10 页
primary 完成 查找！
primary 查找 全部产品列表 第 11 页
primary 完成 查找！
primary 查找 全部产品列表 第 12 页
primary 完成 查找！
primary 查找 全部产品列表 第 13 页
primary 完成 查找！
primary 查找 全部产品列表 第 14 页
primary 完成 查找！
primary 查找 全部产品列表 第 15 页
primary 完成 查找！
primary 查找 全部产品列表 第 16 页
primary 完成 查找！
primary 查找 全部产品列表 第 17 页
primary 完成 查找！
primary 查找 全部产品列表 第 18 页
primary 完成 查找！
primary 查找 全部产品列表 第 19 页
primary 完成 查找！
primary 查找 全部产品列表 第 20 页
primary 完成 查找！
primary 查找 全部产品列表 第 21 页
primary 完成 查找！
primary 查找 全部产品列表 第 22 页
primary 完成 查找！
primary 查找 全部产品列表 第 23 页
primary 完成 查找！
primary 查找 全部产品列表 第 24 页
primary 完

In [9]:
len(products)

818

In [14]:
%store products

Stored 'products' (list)


In [3]:
%store -r products
len(products)

818

In [4]:
successed = {}
excepted = {}
running_tasks = {}

In [35]:
# start-process celery -ArgumentList "worker -A libs.CeleryTasks.tasks -c 4 -l info -Q Eyelashes_PM,celery -n Eyelashes[alibaba]@localhost"
# data = {'price': {'isTieredPricing': False, 'price_range':[0.8, 1.1]}}

data = {}
data['product_detail'] = {}
data['product_detail']['fp'] = {}
data['product_detail']['lp'] = '<img src="//sc01.alicdn.com/kf/HTB15JbiB2iSBuNkSnhJq6zDcpXaX/231186930/HTB15JbiB2iSBuNkSnhJq6zDcpXaX.jpg" alt="faq and contact" ori-width="750" ori-height="927" />'

model = '51c'

count = 0
for p in products:
    ali_id = p['ali_id']
    if ali_id in excepted:
        continue
    if ali_id in successed:
        continue
        
    if p['pid'].lower().startswith(model):
        count += 1
        print(count, ':')
        data['ali_id'] = ali_id
        running_tasks[ali_id] = tasks.alibaba_update_product.apply_async(args=[data], queue='Eyelashes_PM')
        
        while True:
            finished = []
            for key in running_tasks:
                task_result = running_tasks[key]
                if task_result.ready():
                    print(task_result.get())
                    if task_result.get()[1]:
                        successed[key] = p
                    finished.append(key)
                    
            for key in finished:
                print(key, running_tasks)
                del running_tasks[key]
            
            if len(running_tasks.keys()) == 4:
                print('wait and try ..., ')
                time.sleep(1)
                continue
            else:
                print('')
                break

1 :
['60765052269', False]
['60765051493', False]
['60765052294', False]
60765052269 {'60765052269': <AsyncResult: 0a1ff19b-1a29-4f0b-a5ee-d2cfb30d2684>, '60765051493': <AsyncResult: 6e951130-3d57-4c62-b076-f63639fce82e>, '60765052294': <AsyncResult: a0a49eda-2bc1-4d01-b1bb-51f5b6585d6e>, '60761678736': <AsyncResult: de9e69f0-c861-4654-af65-01f6b9342092>}
60765051493 {'60765051493': <AsyncResult: 6e951130-3d57-4c62-b076-f63639fce82e>, '60765052294': <AsyncResult: a0a49eda-2bc1-4d01-b1bb-51f5b6585d6e>, '60761678736': <AsyncResult: de9e69f0-c861-4654-af65-01f6b9342092>}
60765052294 {'60765052294': <AsyncResult: a0a49eda-2bc1-4d01-b1bb-51f5b6585d6e>, '60761678736': <AsyncResult: de9e69f0-c861-4654-af65-01f6b9342092>}

2 :

3 :

4 :
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
wait and try ..., 
['60761678736', False]
['60763079692', False]
60761678736 {'60761678736': <AsyncResult: de9e69f0-c861-4654-af

In [13]:
successed

{}

In [55]:
data = {'ali_id':'60765452670'}
data['product_detail'] = {}
data['product_detail']['fp'] = {}
data['product_detail']['lp'] = '<img src="//sc01.alicdn.com/kf/HTB15JbiB2iSBuNkSnhJq6zDcpXaX/231186930/HTB15JbiB2iSBuNkSnhJq6zDcpXaX.jpg" alt="faq and contact" ori-width="750" ori-height="927" />'

tasks.alibaba_update_product.apply_async(args=[data], queue='Eyelashes_PM')

<AsyncResult: acf1b3d1-78e0-44f4-aba4-bc53fec5c6f0>

['4556', '3245']